In [68]:
import pandas as pd
from sqlalchemy import create_engine

### Extract CSVs into DataFrames

In [69]:
premise_file = "../Resources/LicensePremise.csv"
premise_data = pd.read_csv(premise_file)
premise_data.head()


,License Serial Number,Premises Name,License Certificate Number,License Received Date,County ID Code
0,1311660,CANA ARRIBA GROCERY NO 2 INC,NaN,2018-06-29T00:00:00,0
1,1310023,JOHANA GROCERY & DELI CORP,NaN,2018-04-27T00:00:00,0
2,1310024,TKO BEVERAGES LLC,NaN,2018-04-27T00:00:00,1
3,1310024,TKO BEVERAGES LLC,NaN,2018-04-27T00:00:00,1
4,1311663,181 LEXINGTON AVENUE BBQ LLC,NaN,2018-06-29T00:00:00,2


In [70]:
county_file = "../Resources/CountyLicenseCount.csv"
county_data = pd.read_csv(county_file)
county_data.head()


,ID,County Name (Licensee),County ID Code,License Count
0,0,ALBANY,5,77
1,1,ALLEGANY,59,4
2,2,BRONX,0,104
3,3,BROOME,35,14
4,4,CATTARAUGUS,41,9


### Transform premise DataFrame

In [71]:
# Copy only the columns needed into a new DataFrame.
new_premise_data = premise_data[['License Serial Number', 'Premises Name', 'County ID Code']]

# Rename columns to fit the tables created in the database.
new_premise_data = new_premise_data.rename(columns={'License Serial Number':'id', 'Premises Name':'premise_name', 'County ID Code':'county_id'})

# Handle any duplicates. HINT: some locations have the same name but each license number is unique.
new_premise_data = new_premise_data.drop_duplicates(subset='id', keep='first')

# Set index to the previously created primary key.
new_premise_data.set_index('id')

# display the frame
new_premise_data.head()

,id,premise_name,county_id
0,1311660,CANA ARRIBA GROCERY NO 2 INC,0
1,1310023,JOHANA GROCERY & DELI CORP,0
2,1310024,TKO BEVERAGES LLC,1
4,1311663,181 LEXINGTON AVENUE BBQ LLC,2
5,1310029,AZIZ DELI & GRILL CORP,1


### Transform county DataFrame

In [72]:
# Copy only the columns needed into a new DataFrame.
# NOTE: We need all the columns

# Rename columns to fit the tables created in the database.
new_county_data = county_data.rename(columns={'ID':'id', 'County Name (Licensee)':'county_name', 'License Count':'license_count'  ,'County ID Code':'county_id'})

# Handle any duplicates. HINT: some locations have the same name but each license number is unique.
new_county_data = new_county_data.drop_duplicates(subset='id', keep='first')

# Set index to the previously created primary key.
new_county_data.set_index('id')

# display the frame
new_county_data.head()

,id,county_name,county_id,license_count
0,0,ALBANY,5,77
1,1,ALLEGANY,59,4
2,2,BRONX,0,104
3,3,BROOME,35,14
4,4,CATTARAUGUS,41,9


### Create database connection

In [73]:
rds_connection_string = "postgres:postgres@localhost:5432/customer_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [74]:
# Confirm tables
engine.table_names()

['premise', 'county']

### Load DataFrames into database

In [76]:
# Premise Data
new_premise_data.to_sql(name='premise', con=engine, if_exists='append', index=False)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "premise_pkey"
DETAIL:  Key (id)=(1311660) already exists.

[SQL: INSERT INTO premise (id, premise_name, county_id) VALUES (%(id)s, %(premise_name)s, %(county_id)s)]
[parameters: ({'id': 1311660, 'premise_name': 'CANA ARRIBA GROCERY NO 2 INC', 'county_id': 0}, {'id': 1310023, 'premise_name': 'JOHANA GROCERY & DELI CORP', 'county_id': 0}, {'id': 1310024, 'premise_name': 'TKO BEVERAGES LLC', 'county_id': 1}, {'id': 1311663, 'premise_name': '181 LEXINGTON AVENUE BBQ LLC', 'county_id': 2}, {'id': 1310029, 'premise_name': 'AZIZ DELI & GRILL CORP', 'county_id': 1}, {'id': 2213240, 'premise_name': 'COOPERSTOWN BERT CORP', 'county_id': 3}, {'id': 2213242, 'premise_name': 'DALE B HOLDERMAN', 'county_id': 4}, {'id': 1300091, 'premise_name': 'TWO THOUSAND FIFTEEN ARTISANAL LLC', 'county_id': 2}  ... displaying 10 of 1839 total bound parameter sets ...  {'id': 1311943, 'premise_name': 'DAIRY SHACK LLC'

In [ ]:
# County Data
new_county_data.to_sql(name='county', con=engine, if_exists='append', index=False)